# Queue Wait-Time Tracking Notebook
This notebook uses zone counter and person identification techniques that tracks people standing in a queue and calculates waiting times.
We collect following stats:
- **Waiting zone detection** for monitoring queue duration.
- **Per-person tracking** with unique IDs and live wait time calculation.
- **Bounding box visualization** with overlayed wait times.
- **Automatic handling** of entry/exit to calculate accurate wait durations.
- **Real-time JSON stats** output including:
  - Current timestamp
  - Live people with their wait times
  - Average wait time of completed people
  - Last 5 recorded wait times
  - Total processed people

# Main Function to Run Wait-Time Tracking

- **Input Video:** Provide the source file path.  
- **Waiting Zone Polygon:** Manually define the waiting area.  
- **Model Loading:** Load model for person detection.  
- **Tracking Loop:**  
  - Detect people and assign unique IDs.  
  - Track waiting durations per person inside the polygon.  
  - Handle transitions when a person enters or exits the waiting zone.  
  - Visualize bounding boxes with live wait times on the video.  
- **Queue Stats:** Periodically calculate and display JSON stats for monitoring wait times.


In [ ]:
! pip install degirum degirum_tools -q

In [ ]:
import time
import cv2
import numpy as np
import degirum as dg
import degirum_tools
from collections import deque

video_source = '/content/video.mp4'
polygon_waiting = np.array([[200, 150], [850, 150],  [850, 600],[200, 600]], np.int32)

model = dg.load_model(
    model_name="yolov8s_coco--640x640_quant_hailort_multidevice_1",
    inference_host_address="@local",
    zoo_url="degirum/hailo"
    )

# Helpers
def get_center(bbox):
    x1, y1, x2, y2 = bbox
    return (int((x1+x2)/2), int((y1+y2)/2))

def point_in_polygon(pt, polygon):
    return cv2.pointPolygonTest(polygon, pt, False) >= 0

# -----------------------------
# Tracking
# -----------------------------
tracked = {}  # {pid: {enter_time, wait_time, done}}
next_pid = 1
completed_waits = []  # list of wait times
last_5 = deque(maxlen=5)
cap = cv2.VideoCapture(video_source)
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()

frame_idx = 0
last_print_time = 0  # Track when we last printed stats
PRINT_INTERVAL = 10  # Print every 10 seconds

# Main loop
with degirum_tools.Display("Wait Tracker") as disp:
    for result in degirum_tools.predict_stream(model, video_source):
        now = frame_idx / fps  # use video time instead of wall clock
        frame_idx += 1
        frame = result.image.copy()
        detections = result.results

        current_people = {}

        for det in detections:
            if det["label"].lower() != "person" or det["score"] <= 0.35:
                continue

            bbox = det["bbox"]
            center = get_center(bbox)

            in_zone = point_in_polygon(center, polygon_waiting)

            # Assign pid
            pid = None
            for tid, data in tracked.items():
                if abs(center[0] - data["center"][0]) < 50 and abs(center[1] - data["center"][1]) < 50:
                    pid = tid
                    break
            if pid is None:
                pid = next_pid
                next_pid += 1
                tracked[pid] = {"enter_time": None, "wait_time": 0, "done": False, "center": center}

            tracked[pid]["center"] = center

            # Zone logic
            if in_zone and tracked[pid]["enter_time"] is None:
                tracked[pid]["enter_time"] = now

            if not in_zone and tracked[pid]["enter_time"] is not None and not tracked[pid]["done"]:
                wait_time = now - tracked[pid]["enter_time"]
                tracked[pid]["wait_time"] = wait_time
                tracked[pid]["done"] = True
                completed_waits.append(wait_time)
                last_5.append(wait_time)

            if in_zone and not tracked[pid]["done"]:
                tracked[pid]["wait_time"] = now - tracked[pid]["enter_time"]

            current_people[pid] = round(tracked[pid]["wait_time"], 1)

            # Draw
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])),
                          (int(bbox[2]), int(bbox[3])), (0,255,255), 2)
            cv2.putText(frame, f"P{pid}:{tracked[pid]['wait_time']:.1f}s",
                        (int(bbox[0]), int(bbox[1]-5)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

        # Print stats every 10 seconds
        if now - last_print_time >= PRINT_INTERVAL:
            avg_wait = round(sum(completed_waits)/len(completed_waits), 1) if completed_waits else 0

            print("===="*10)
            print(f"timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"video_time: {now:.1f}s")
            print(f"people:{current_people}")
            print(f"avg_wait_time: {avg_wait}")
            print(f"last_5_wait_times: {list(last_5)}")
            print(f"total_processed_people: {len(completed_waits)}\n")
            
            last_print_time = now

        # Draw polygon
        cv2.polylines(frame, [polygon_waiting], True, (0,0,255), 2)

        disp.show(frame)